# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

In [3]:
# Requisitos: pandas, numpy
import pandas as pd
import numpy as np

# 0) Cargar datos (desde la URL dada)
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

# Inspección rápida (muestra columnas)
print("Columnas:", df.columns.tolist())
print(df.shape)


Columnas: ['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value', 'Response', 'Coverage', 'Education', 'Effective To Date', 'EmploymentStatus', 'Gender', 'Income', 'Location Code', 'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim', 'Months Since Policy Inception', 'Number of Open Complaints', 'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type', 'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size', 'Vehicle Type']
(10910, 26)


In [4]:
# Asegúrate del nombre exacto de las columnas:
# Ajustes automáticos si hay espacios o mayúsculas diferentes
cols = {c: c.strip() for c in df.columns}
df.rename(columns=cols, inplace=True)

# Nombres comunes esperados:
# - 'total_claim_amount'  (float)
# - 'response'            (string: 'Yes'/'No')
# Si son distintos, adapta los nombres siguientes.
total_col = [c for c in df.columns if 'total' in c.lower() and 'claim' in c.lower()]
resp_col  = [c for c in df.columns if c.lower() == 'response']

if not total_col or not resp_col:
    raise ValueError("No encuentro columnas 'total_claim_amount' o 'response'. Revisa los nombres de columnas.")

total_col = total_col[0]
resp_col = resp_col[0]

# Convertir total_claim_amount a numérico si hace falta
df[total_col] = pd.to_numeric(df[total_col], errors='coerce')

# Filtrado
low_claim_yes = df[(df[total_col] < 1000) & (df[resp_col].str.strip().str.lower() == 'yes')].copy()

print("Clientes con total_claim_amount < 1000 y respuesta 'Yes':", len(low_claim_yes))
low_claim_yes.head()


Clientes con total_claim_amount < 1000 y respuesta 'Yes': 1399


,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
8,8,FM55990,California,5989.773931,Yes,Premium,College,1/19/11,Employed,M,...,0.0,1,Personal Auto,Personal L1,Offer2,Branch,739.200000,Sports Car,Medsize,NaN
15,15,CW49887,California,4626.801093,Yes,Basic,Master,1/16/11,Employed,F,...,0.0,1,Special Auto,Special L1,Offer2,Branch,547.200000,SUV,Medsize,NaN
19,19,NJ54277,California,3746.751625,Yes,Extended,College,2/26/11,Employed,F,...,1.0,1,Personal Auto,Personal L2,Offer2,Call Center,19.575683,Two-Door Car,Large,A
27,27,MQ68407,Oregon,4376.363592,Yes,Premium,Bachelor,2/28/11,Employed,F,...,0.0,1,Personal Auto,Personal L3,Offer2,Agent,60.036683,Four-Door Car,Medsize,NaN


2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

In [5]:
# Detectar columnas relevantes (ser robusto si nombres varían)
def find_col(part1, part2=None):
    for c in df.columns:
        name = c.lower()
        if part2:
            if part1 in name and part2 in name:
                return c
        else:
            if part1 in name:
                return c
    return None

monthly_col = find_col('monthly', 'premium') or find_col('monthly')
clv_col     = find_col('customer', 'lifetime') or find_col('customer')  # 'customer_lifetime_value'
policy_col  = find_col('policy', 'type') or find_col('policy')
gender_col  = find_col('gender')

if not monthly_col or not clv_col or not policy_col or not gender_col:
    print("Atención: alguna columna no encontrada automáticamente. Revisa manualmente names:", monthly_col, clv_col, policy_col, gender_col)

# Filtrar solo quienes respondieron Yes
responders = df[df[resp_col].str.strip().str.lower() == 'yes'].copy()

# Asegurar numéricos
responders[monthly_col] = pd.to_numeric(responders[monthly_col], errors='coerce')
responders[clv_col] = pd.to_numeric(responders[clv_col], errors='coerce')
responders[total_col] = pd.to_numeric(responders[total_col], errors='coerce')

# Agrupar por policy_type y gender y calcular medias
agg = responders.groupby([policy_col, gender_col]).agg(
    avg_monthly_premium = (monthly_col, 'mean'),
    avg_customer_lifetime_value = (clv_col, 'mean'),
    avg_total_claim_amount = (total_col, 'mean'),
    count = (resp_col, 'size')
).reset_index()

# Mostrar ordenado por contador descendente
agg = agg.sort_values('count', ascending=False)
pd.set_option('display.float_format', lambda x: f"{x:,.2f}")
agg.head(30)


,Policy Type,Gender,avg_monthly_premium,avg_customer_lifetime_value,avg_total_claim_amount,count
2,Personal Auto,F,99.00,"8,339.79",452.97,540
3,Personal Auto,M,91.09,"7,448.38",457.01,536
0,Corporate Auto,F,94.30,"7,712.63",433.74,169
1,Corporate Auto,M,92.19,"7,944.47",408.58,154
4,Special Auto,F,92.31,"7,691.58",453.28,35
5,Special Auto,M,86.34,"8,247.09",429.53,32


3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [6]:
state_col = find_col('state') or find_col('region') or find_col('statecode')
if not state_col:
    raise ValueError("No encuentro columna de 'state'. Revisa nombres de columnas.")

counts_by_state = df[state_col].value_counts().rename_axis(state_col).reset_index(name='n_customers')
states_over_500 = counts_by_state[counts_by_state['n_customers'] > 500].copy()

print("Estados con > 500 clientes:", len(states_over_500))
states_over_500


Estados con > 500 clientes: 5


,State,n_customers
0,California,3552
1,Oregon,2909
2,Arizona,1937
3,Nevada,993
4,Washington,888


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [7]:
edu_col = find_col('education') or find_col('edu')
if not edu_col:
    raise ValueError("No encuentro columna 'education'.")

# Asegurar CLV numérico
df[clv_col] = pd.to_numeric(df[clv_col], errors='coerce')

clv_stats = df.groupby([edu_col, gender_col])[clv_col].agg(['max', 'min', 'median', 'count']).reset_index()
clv_stats.rename(columns={'max':'CLV_max','min':'CLV_min','median':'CLV_median','count':'n'}, inplace=True)
clv_stats.sort_values(['CLV_median'], ascending=False).head(50)


,Education,Gender,CLV_max,CLV_min,CLV_median,n
7,High School or Below,M,"83,325.38","1,940.98","6,286.73",1556
6,High School or Below,F,"55,277.45","2,144.92","6,039.55",1590
3,College,M,"61,134.68","1,918.12","6,005.85",1596
8,Master,F,"51,016.07","2,417.78","5,729.86",475
0,Bachelor,F,"73,225.96","1,904.00","5,640.51",1700
2,College,F,"61,850.19","1,898.68","5,623.61",1608
9,Master,M,"50,568.26","2,272.31","5,579.10",412
5,Doctor,M,"32,677.34","2,267.60","5,577.67",201
1,Bachelor,M,"67,907.27","1,898.01","5,548.03",1572
4,Doctor,F,"44,856.11","2,395.57","5,332.46",200


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [8]:
# Detectar columna de fecha (cualquier columna que contenga 'date' o 'bind')
date_col = None
for c in df.columns:
    if 'date' in c.lower() or 'bind' in c.lower() or 'month' in c.lower():
        date_col = c
        break

if date_col is None:
    raise ValueError("No encuentro columna de fecha. Ajusta manualmente el nombre de la columna de fecha.")

# Convertir a datetime
df[date_col] = pd.to_datetime(df[date_col], errors='coerce')

# Crear campo mes (formato YYYY-MM o nombre de mes)
df['sale_month'] = df[date_col].dt.to_period('M').astype(str)  # e.g., '2020-05'

# Pivot table: rows = states, columns = months
policies_by_state_month = df.groupby([state_col, 'sale_month']).size().reset_index(name='n_policies')
pivot = policies_by_state_month.pivot(index=state_col, columns='sale_month', values='n_policies').fillna(0).astype(int)

# Ordenar columnas cronológicamente (por string YYYY-MM funciona)
pivot = pivot.reindex(sorted(pivot.columns), axis=1)

pivot.head(20)


C:\Users\deysi\AppData\Local\Temp\ipykernel_17964\3107881687.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[date_col] = pd.to_datetime(df[date_col], errors='coerce')


sale_month,2011-01,2011-02
State,,
Arizona,1008,929
California,1918,1634
Nevada,551,442
Oregon,1565,1344
Washington,463,425


6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [9]:
# Calcular total de pólizas por estado
totals = df[state_col].value_counts().reset_index()
totals.columns = [state_col, 'total_policies']
top3_states = totals.head(3)[state_col].tolist()

# Filtrar pivot para esos 3 estados
top3_pivot = pivot.loc[top3_states].copy()
top3_pivot


sale_month,2011-01,2011-02
State,,
California,1918,1634
Oregon,1565,1344
Arizona,1008,929


7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [10]:
# your code goes here

# Detectar columnas de canal: columnas con pocos valores únicos y que no sean 'response' ni 'gender', etc.
candidate_channels = []
for c in df.columns:
    if c == resp_col: 
        continue
    nunique = df[c].nunique(dropna=False)
    if nunique <= 3:  # binarias o ternarias
        # comprobar si contiene 'yes'/'no' o 0/1
        vals = set(str(x).strip().lower() for x in df[c].dropna().unique())
        if {'yes','no'}.issubset(vals) or vals.issubset({'0','1','0.0','1.0'}) or vals.issubset({'0','1'}):
            candidate_channels.append(c)

print("Canales candidatos:", candidate_channels)

# Si se identifican canales, hacemos melt; sino, pide al usuario indicar columnas.
if not candidate_channels:
    print("No se detectaron columnas de canal automáticamente. Proporciona los nombres de las columnas de canal (p.ej. ['email','sms','phone']).")
else:
    # Creamos DataFrame 'long' donde cada fila = cliente x canal con indicador si fue contactado
    channels = candidate_channels
    long = df[[resp_col] + channels].melt(id_vars=[resp_col], value_vars=channels,
                                         var_name='channel', value_name='contacted')

    # Normalizar valores 'yes'/'no' etc a booleans
    long['contacted_bool'] = long['contacted'].astype(str).str.strip().str.lower().map({'yes': True, '1': True, '1.0': True, 'true': True}).fillna(False)

    # Calcular tasa de respuesta por canal: among contacted customers, proportion with response Yes
    long_with_response = long.copy()
    # Necesitamos agregar por canal: responders_among_contacted / total_contacted
    channel_stats = long_with_response.groupby('channel').apply(
        lambda g: pd.Series({
            'n_contacted': g['contacted_bool'].sum(),
            'n_responded': ((g['contacted_bool']) & (g[resp_col].astype(str).str.strip().str.lower() == 'yes')).sum()
        })
    ).reset_index()

    channel_stats['response_rate'] = channel_stats['n_responded'] / channel_stats['n_contacted'].replace(0, np.nan)
    channel_stats = channel_stats.sort_values('response_rate', ascending=False).reset_index(drop=True)
    channel_stats


Canales candidatos: []
No se detectaron columnas de canal automáticamente. Proporciona los nombres de las columnas de canal (p.ej. ['email','sms','phone']).
